# TorontonianLib

A Fortran library wrapped into Python to generate samples from Torontonians

### Using the library

This libary requires f2py to be installed. 

To compile simply run "make library" from terminal, or run the following cell

In [1]:
%%capture
!make library;

After compilation import the TorontonianSamples library 

In [1]:
import torontonian_samples as tor

Import numpy and random

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats=['svg']
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } "
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
np.set_printoptions(linewidth=200)

Import the Gaussian backend of the strawberryfields package

In [3]:
import strawberryfields.backends.gaussianbackend.gaussiancircuit as gc

Create a 2-mode Gaussian state 

In [4]:
state = gc.GaussianModes(2,hbar=2)
s=np.arcsinh(0.5)
state.squeeze(s,0,0)
state.squeeze(-s,0,1)
state.beamsplitter(np.pi/4,0,0,1)


The following function generates random Haar unitaries of dimensions $n$

In [5]:
from scipy import diagonal, randn
from scipy.linalg import qr
def haar_measure(n):
    '''A Random matrix distributed with Haar measure
    See https://arxiv.org/abs/math-ph/0609050
    How to generate random matrices from the classical compact groups
    by Francesco Mezzadri '''
    z = (randn(n,n) + 1j*randn(n,n))/np.sqrt(2.0)
    q,r = qr(z)
    d = diagonal(r)
    ph = d/np.abs(d)
    q = np.multiply(q,ph,q)
    return q

Using the Gaussian state and the random unitary defined above, create an $l$ mode Gaussian state with covriance matrix "tormat"

In [6]:
l =10
U = haar_measure(l)
state = gc.GaussianModes(l,hbar=2)
s = np.arcsinh(1.0)
for i in range(l):
    state.squeeze(s,0,i)
state.apply_u(U)
gbsmat = state.Amat()
tormat = state.scovmat()

Define the mean vector of the Gaussian state. Usually mean is taken to be zero. 

In [7]:
r = np.zeros(tormat.shape[0])

Call the torontoniansample, a fortran subroutine as follows to generate a sample: 



In [8]:
random.randint(0,10**6)

38173

In [9]:
tor.torontoniansamples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l)

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int64)

Now the above function can be called multiple times in a loop to generate as many samples as your heart desires.

In [12]:
samples=[]
for i in range(20):
    tmp = list(tor.torontoniansamples.generatesample(covmat=tormat,mean=r,seed=random.randint(0,10**6),n_sample=l))
    samples.append(tmp)

In [13]:
samples

[[1, 1, 0, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 0, 1, 0, 1, 1, 1, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 1, 1]]

Save the samples in a file

In [11]:
np.savetxt("sample.dat",samples, fmt='%i')